https://github.com/wesm/pydata-book

In [2]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas import Series, DataFrame
import seaborn as sns

# used for example for random
from numpy import *
# for matplot
%matplotlib inline

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
from IPython.display import display
from IPython.display import Image

Much of the programming work in data analysis and modeling is spent on data preparation:
loading, cleaning, transforming, and rearranging.

# Combining and Merging Data Sets

Data contained in pandas objects can be combined together in a number of built-in
ways:
<br>• pandas.merge connects rows in DataFrames based on one or more keys. This will
be familiar to users of SQL or other relational databases, as it implements database
join operations.
<br>• pandas.concat glues or stacks together objects along an axis.
<br>• combine_first instance method enables splicing together overlapping data to fill
in missing values in one object with values from another.
I will address each of these and give a number of examples. They’ll be utilized in examples
throughout the rest of the book.

## Database-style DataFrame Merges

Merge or join operations combine data sets by linking rows using one or more keys.
These operations are central to relational databases. The merge function in pandas is
the main entry point for using these algorithms on your data.

https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html

Pandas documentation on Merge, join and concatanate:
https://pandas.pydata.org/pandas-docs/stable/merging.html

In [12]:
df1 = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                 'data1': range(7)})
df2 = DataFrame({'key': ['a', 'b', 'd'],
                 'data2': range(3)})

df1
df2

#many-to-one merge situation
pd.merge(df1, df2)

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


,data2,key
0,0,a
1,1,b
2,2,d


,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


In [13]:
#Note that I didn’t specify which column to join on. If not specified, merge uses the
#overlapping column names as the keys.

pd.merge(df1, df2, on = 'key')

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


In [9]:
#If the column names are different in each object, you can specify them separately:
df3 = DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'], 'data1': range(7)})

df4 = DataFrame({'rkey':['a', 'b', 'd'], 'data2': range(3)})

df3
df4

pd.merge(df3, df4, left_on= 'lkey', right_on = 'rkey')

,data1,lkey
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


,data2,rkey
0,0,a
1,1,b
2,2,d


,data1,lkey,data2,rkey
0,0,b,1,b
1,1,b,1,b
2,6,b,1,b
3,2,a,0,a
4,4,a,0,a
5,5,a,0,a


You probably noticed that the 'c' and 'd' values and associated data are missing from
the result. By default merge does an <font color='red'> **'inner' join** </font>; the keys in the result are the <font color='red'>** intersection.**</font>
Other possible options are 
```python
'left', 'right', and 'outer'
```
The outer join takes the
union of the keys, combining the effect of applying both left and right joins:

In [14]:
pd.merge(df1, df2, how = 'outer')

,data1,key,data2
0,0.0,b,1.0
1,1.0,b,1.0
2,6.0,b,1.0
3,2.0,a,0.0
4,4.0,a,0.0
5,5.0,a,0.0
6,3.0,c,NaN
7,NaN,d,2.0


In [16]:
#Many-to-many merges have well-defined though not necessarily intuitive behavior.
#Many-to-many joins form the Cartesian product of the rows. Since there were 3 'b'
#rows in the left DataFrame and 2 in the right one, there are 6 'b' rows in the result.
df1 = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
    'data1': range(6)})
df2 = DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
    'data2': range(5)})

df1
df2

pd.merge(df1, df2, on = 'key', how = 'left')

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,b


,data2,key
0,0,a
1,1,b
2,2,a
3,3,b
4,4,d


,data1,key,data2
0,0,b,1.0
1,0,b,3.0
2,1,b,1.0
3,1,b,3.0
4,2,a,0.0
5,2,a,2.0
6,3,c,NaN
7,4,a,0.0
8,4,a,2.0
9,5,b,1.0


In [17]:
pd.merge(df1,df2, how = 'inner')

,data1,key,data2
0,0,b,1
1,0,b,3
2,1,b,1
3,1,b,3
4,5,b,1
5,5,b,3
6,2,a,0
7,2,a,2
8,4,a,0
9,4,a,2


In [19]:
#To merge with multiple keys, pass a list of column names:
left = DataFrame({'key1': ['foo', 'foo', 'bar'],
    'key2': ['one', 'two', 'one'],
    'lval': [1, 2, 3]})
right = DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
    'key2': ['one', 'one', 'one', 'two'],
    'rval': [4, 5, 6, 7]})

left
right

pd.merge(left,right, on = ['key1', 'key2'], how = 'outer')


,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


To determine which key combinations will appear in the result depending on the choice
of merge method, think of the multiple keys as forming an array of tuples to be used
as a single join key (even though it’s not actually implemented that way).
<img src = "IMG/pd_merge.png">

In [24]:
pd.merge(left, right, on = 'key1')

pd.merge(left, right, on='key1', suffixes = ('_left', '_right'))

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [38]:


A = DataFrame({'lkey':['foo', 'bar', 'baz', 'foo'], 'value':range(1,5) })
A

B = DataFrame({'rkey':['foo', 'bar', 'qux', 'bar'], 'value':range(5,9)})
B
A.merge(B, left_on ='lkey', right_on = 'rkey', how = 'outer')
A.merge(B,  left_index=True, right_index=True, how = 'outer')
#A.merge(B,  left_index=True, how = 'outer')

,lkey,value
0,foo,1
1,bar,2
2,baz,3
3,foo,4


,rkey,value
0,foo,5
1,bar,6
2,qux,7
3,bar,8


,lkey,value_x,rkey,value_y
0,foo,1.0,foo,5.0
1,foo,4.0,foo,5.0
2,bar,2.0,bar,6.0
3,bar,2.0,bar,8.0
4,baz,3.0,NaN,NaN
5,NaN,NaN,qux,7.0


,lkey,value_x,rkey,value_y
0,foo,1,foo,5
1,bar,2,bar,6
2,baz,3,qux,7
3,foo,4,bar,8


https://chrisalbon.com/python/pandas_join_merge_dataframe.html

In [141]:
raw_data = {
        'subject_id': ['1', '2', '3', '4', '5'],
        'first_name': ['Alex', 'Amy', 'Allen', 'Alice', 'Ayoung'], 
        'last_name': ['Anderson', 'Ackerman', 'Ali', 'Aoni', 'Atiches']}
df_a = pd.DataFrame(raw_data, columns = ['subject_id', 'first_name', 'last_name'])
df_a

raw_data = {
        'subject_id': ['4', '5', '6', '7', '8'],
        'first_name': ['Billy', 'Brian', 'Bran', 'Bryce', 'Betty'], 
        'last_name': ['Bonder', 'Black', 'Balwner', 'Brice', 'Btisan']}
df_b = pd.DataFrame(raw_data, columns = ['subject_id', 'first_name', 'last_name'])
df_b

raw_data = {
        'subject_id': ['1', '2', '3', '4', '5', '7', '8', '9', '10', '11'],
        'test_id': [51, 15, 15, 61, 16, 14, 15, 1, 61, 16]}
df_n = pd.DataFrame(raw_data, columns = ['subject_id','test_id'])
df_n

df_a.dtypes
df_b.dtypes
df_n.dtypes


,subject_id,first_name,last_name
0,1,Alex,Anderson
1,2,Amy,Ackerman
2,3,Allen,Ali
3,4,Alice,Aoni
4,5,Ayoung,Atiches


,subject_id,first_name,last_name
0,4,Billy,Bonder
1,5,Brian,Black
2,6,Bran,Balwner
3,7,Bryce,Brice
4,8,Betty,Btisan


,subject_id,test_id
0,1,51
1,2,15
2,3,15
3,4,61
4,5,16
5,7,14
6,8,15
7,9,1
8,10,61
9,11,16


subject_id    object
first_name    object
last_name     object
dtype: object

subject_id    object
first_name    object
last_name     object
dtype: object

subject_id    object
test_id        int64
dtype: object

<img src = 'IMG/pd_merge_ex.png'>

## Join the two dataframes along rows

In [142]:
df_new = pd.concat([df_a, df_b])
df_new

,subject_id,first_name,last_name
0,1,Alex,Anderson
1,2,Amy,Ackerman
2,3,Allen,Ali
3,4,Alice,Aoni
4,5,Ayoung,Atiches
0,4,Billy,Bonder
1,5,Brian,Black
2,6,Bran,Balwner
3,7,Bryce,Brice
4,8,Betty,Btisan


## Join the two dataframes along columns

In [143]:
pd.concat([df_a, df_b], axis = 1)

,subject_id,first_name,last_name,subject_id,first_name,last_name
0,1,Alex,Anderson,4,Billy,Bonder
1,2,Amy,Ackerman,5,Brian,Black
2,3,Allen,Ali,6,Bran,Balwner
3,4,Alice,Aoni,7,Bryce,Brice
4,5,Ayoung,Atiches,8,Betty,Btisan


## Merge two dataframes along the subject_id value

In [144]:
pd.merge(df_new, df_n, on = 'subject_id')

,subject_id,first_name,last_name,test_id
0,1,Alex,Anderson,51
1,2,Amy,Ackerman,15
2,3,Allen,Ali,15
3,4,Alice,Aoni,61
4,4,Billy,Bonder,61
5,5,Ayoung,Atiches,16
6,5,Brian,Black,16
7,7,Bryce,Brice,14
8,8,Betty,Btisan,15


## Merge with outer join

"Full outer join produces the set of all records in Table A and Table B, with matching records from both sides where available. If there is no match, the missing side will contain null." - source

In [149]:
pd.merge(df_a, df_b, on = 'subject_id', how = 'outer')
pd.merge(df_a, df_b, how = 'outer')

,subject_id,first_name_x,last_name_x,first_name_y,last_name_y
0,1,Alex,Anderson,NaN,NaN
1,2,Amy,Ackerman,NaN,NaN
2,3,Allen,Ali,NaN,NaN
3,4,Alice,Aoni,Billy,Bonder
4,5,Ayoung,Atiches,Brian,Black
5,6,NaN,NaN,Bran,Balwner
6,7,NaN,NaN,Bryce,Brice
7,8,NaN,NaN,Betty,Btisan


,subject_id,first_name,last_name
0,1,Alex,Anderson
1,2,Amy,Ackerman
2,3,Allen,Ali
3,4,Alice,Aoni
4,5,Ayoung,Atiches
5,4,Billy,Bonder
6,5,Brian,Black
7,6,Bran,Balwner
8,7,Bryce,Brice
9,8,Betty,Btisan


## Merge with inner join

"Inner join produces only the set of records that match in both Table A and Table B." - source

In [153]:
pd.merge(df_a, df_b, on ='subject_id', how = 'inner')
pd.merge(df_a, df_b, on ='subject_id') # as defaullt of how is inner we can skip it.
pd.merge(df_a, df_b, how = 'inner')
pd.merge(df_a, df_b)

,subject_id,first_name_x,last_name_x,first_name_y,last_name_y
0,4,Alice,Aoni,Billy,Bonder
1,5,Ayoung,Atiches,Brian,Black


,subject_id,first_name_x,last_name_x,first_name_y,last_name_y
0,4,Alice,Aoni,Billy,Bonder
1,5,Ayoung,Atiches,Brian,Black


,subject_id,first_name,last_name


,subject_id,first_name,last_name


## Merge with right join

In [155]:
pd.merge(df_a, df_b, on = 'subject_id', how='right')

,subject_id,first_name_x,last_name_x,first_name_y,last_name_y
0,4,Alice,Aoni,Billy,Bonder
1,5,Ayoung,Atiches,Brian,Black
2,6,NaN,NaN,Bran,Balwner
3,7,NaN,NaN,Bryce,Brice
4,8,NaN,NaN,Betty,Btisan


## Merge with left join

"Left outer join produces a complete set of records from Table A, with the matching records (where available) in Table B. If there is no match, the right side will contain null." - source


In [157]:
pd.merge(df_a, df_b, on = 'subject_id', how='left')

df_a.merge(df_b, on = 'subject_id', how = 'left')

,subject_id,first_name_x,last_name_x,first_name_y,last_name_y
0,1,Alex,Anderson,NaN,NaN
1,2,Amy,Ackerman,NaN,NaN
2,3,Allen,Ali,NaN,NaN
3,4,Alice,Aoni,Billy,Bonder
4,5,Ayoung,Atiches,Brian,Black


,subject_id,first_name_x,last_name_x,first_name_y,last_name_y
0,1,Alex,Anderson,NaN,NaN
1,2,Amy,Ackerman,NaN,NaN
2,3,Allen,Ali,NaN,NaN
3,4,Alice,Aoni,Billy,Bonder
4,5,Ayoung,Atiches,Brian,Black


## Merge while adding a suffix to duplicate column names


In [158]:
pd.merge(df_a, df_b, on='subject_id', how='left', suffixes=('_left', '_right'))

,subject_id,first_name_left,last_name_left,first_name_right,last_name_right
0,1,Alex,Anderson,NaN,NaN
1,2,Amy,Ackerman,NaN,NaN
2,3,Allen,Ali,NaN,NaN
3,4,Alice,Aoni,Billy,Bonder
4,5,Ayoung,Atiches,Brian,Black


## Merge based on indexes

In some cases, the merge key or keys in a DataFrame will be found in its index. In this
case, you can pass 
```python
left_index=True or right_index=True
```
(or both) to indicate that the
index should be used as the merge key:

In [159]:
pd.merge(df_a, df_b, right_index=True, left_index=True)

,subject_id_x,first_name_x,last_name_x,subject_id_y,first_name_y,last_name_y
0,1,Alex,Anderson,4,Billy,Bonder
1,2,Amy,Ackerman,5,Brian,Black
2,3,Allen,Ali,6,Bran,Balwner
3,4,Alice,Aoni,7,Bryce,Brice
4,5,Ayoung,Atiches,8,Betty,Btisan


```python
DataFrame.set_index(keys, drop=True, append=False, inplace=False, verify_integrity=False)[source]
```
Set the DataFrame index (row labels) using one or more existing columns. By default yields a new object.

In [137]:
raw_data = {
        'subject_id': range(1, 6),
        'first_name': ['Alex', 'Amy', 'Allen', 'Alice', 'Ayoung'], 
        'last_name': ['Anderson', 'Ackerman', 'Ali', 'Aoni', 'Atiches']}
df_a = pd.DataFrame(raw_data, columns = ['subject_id', 'first_name', 'last_name'])
df_a

raw_data = {
        'subject_id': range(6, 11),
        'first_name': ['Billy', 'Brian', 'Bran', 'Bryce', 'Betty'], 
        'last_name': ['Bonder', 'Black', 'Balwner', 'Brice', 'Btisan']}
df_b = pd.DataFrame(raw_data, columns = ['subject_id', 'first_name', 'last_name'])
df_b

raw_data = {
        'subject_id': range(1,11),
        'test_id': [51, 15, 15, 61, 16, 14, 15, 1, 61, 16]}
df_n = pd.DataFrame(raw_data, columns = ['subject_id','test_id'])
df_n

df_a.dtypes
df_b.dtypes
df_n.dtypes


,subject_id,first_name,last_name
0,1,Alex,Anderson
1,2,Amy,Ackerman
2,3,Allen,Ali
3,4,Alice,Aoni
4,5,Ayoung,Atiches


,subject_id,first_name,last_name
0,6,Billy,Bonder
1,7,Brian,Black
2,8,Bran,Balwner
3,9,Bryce,Brice
4,10,Betty,Btisan


,subject_id,test_id
0,1,51
1,2,15
2,3,15
3,4,61
4,5,16
5,6,14
6,7,15
7,8,1
8,9,61
9,10,16


subject_id     int32
first_name    object
last_name     object
dtype: object

subject_id     int32
first_name    object
last_name     object
dtype: object

subject_id    int32
test_id       int64
dtype: object

In [138]:
# Sprawdzam jak można połączyć i zmienić index na subject_id)
merged = pd.merge(df_new, df_n, on = 'subject_id', how = 'outer')
merged
merged = merged.set_index('subject_id')
merged

,subject_id,first_name,last_name,test_id
0,1,Alex,Anderson,51
1,2,Amy,Ackerman,15
2,3,Allen,Ali,15
3,4,Alice,Aoni,61
4,5,Ayoung,Atiches,16
5,6,Billy,Bonder,14
6,7,Brian,Black,15
7,8,Bran,Balwner,1
8,9,Bryce,Brice,61
9,10,Betty,Btisan,16


,first_name,last_name,test_id
subject_id,,,
1,Alex,Anderson,51
2,Amy,Ackerman,15
3,Allen,Ali,15
4,Alice,Aoni,61
5,Ayoung,Atiches,16
6,Billy,Bonder,14
7,Brian,Black,15
8,Bran,Balwner,1
9,Bryce,Brice,61


With hierarchically-indexed data, things are a bit more complicated.
<br>
<br>
In this case, you have to indicate multiple columns to merge on as a list (pay attention
to the handling of duplicate index values):

In [162]:
lefth = DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
    'key2': [2000, 2001, 2002, 2001, 2002],
    'data': np.arange(5.)})

righth = DataFrame(np.arange(12).reshape((6, 2)),
    index=[['Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'],
    [2001, 2000, 2000, 2000, 2001, 2002]],
    columns=['event1', 'event2'])

lefth
righth

pd.merge(lefth, righth, left_on = ['key1', 'key2'], right_index = True)
pd.merge(lefth, righth, left_on = ['key1', 'key2'], right_index = True, how = 'outer')

,data,key1,key2
0,0.0,Ohio,2000
1,1.0,Ohio,2001
2,2.0,Ohio,2002
3,3.0,Nevada,2001
4,4.0,Nevada,2002


event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

,data,key1,key2,event1,event2
0,0.0,Ohio,2000,4,5
0,0.0,Ohio,2000,6,7
1,1.0,Ohio,2001,8,9
2,2.0,Ohio,2002,10,11
3,3.0,Nevada,2001,0,1


,data,key1,key2,event1,event2
0,0.0,Ohio,2000,4.0,5.0
0,0.0,Ohio,2000,6.0,7.0
1,1.0,Ohio,2001,8.0,9.0
2,2.0,Ohio,2002,10.0,11.0
3,3.0,Nevada,2001,0.0,1.0
4,4.0,Nevada,2002,NaN,NaN
4,NaN,Nevada,2000,2.0,3.0


Using the indexes of both sides of the merge is also not an issue:

In [166]:
left2 = DataFrame([[1., 2.], [3., 4.], [5., 6.]], index=['a', 'c', 'e'],
    columns=['Ohio', 'Nevada'])

right2 = DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
    index=['b', 'c', 'd', 'e'], columns=['Missouri', 'Alabama'])
left2
right2

pd.merge(left2, right2, left_index= True, right_index = True, how = 'outer')

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [154]:
help(pd.merge)

Help on function merge in module pandas.core.reshape.merge:

merge(left, right, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), copy=True, indicator=False)
    Merge DataFrame objects by performing a database-style join operation by
    columns or indexes.
    
    If joining columns on columns, the DataFrame indexes *will be
    ignored*. Otherwise if joining indexes on indexes or indexes on a column or
    columns, the index will be passed on.
    
    Parameters
    ----------
    left : DataFrame
    right : DataFrame
    how : {'left', 'right', 'outer', 'inner'}, default 'inner'
        * left: use only keys from left frame, similar to a SQL left outer join;
          preserve key order
        * right: use only keys from right frame, similar to a SQL right outer join;
          preserve key order
        * outer: use union of keys from both frames, similar to a SQL full outer
          join; sort keys lexi

## Join
DataFrame has a more convenient 
```python
pd.join
```
instance for merging by index. It can also be
used to combine together many DataFrame objects having the same or similar indexes
but non-overlapping columns. In the prior example, we could have written:

In [167]:
left2.join(right2, how = 'outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In part for legacy reasons (much earlier versions of pandas), DataFrame’s join method
performs a left join on the join keys. It also supports joining the index of the passed
DataFrame on one of the columns of the calling DataFrame:

In [175]:
left1 = DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
    'value': range(6)})
right1 = DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])

left1
right1
left1.join(right1, on = 'key')

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


,group_val
a,3.5
b,7.0


,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


Lastly, for simple index-on-index merges, you can pass a list of DataFrames to join as
an alternative to using the more general concat function described below:

In [179]:
another = DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
    index=['a', 'c', 'e', 'f'], columns=['New York', 'Oregon'])

another
left2.join([right2, another])

df = left2.join([right2, another], how = 'outer')
df.fillna('brak danych')

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1,2,brak danych,brak danych,7,8
b,brak danych,brak danych,7,8,brak danych,brak danych
c,3,4,9,10,9,10
d,brak danych,brak danych,11,12,brak danych,brak danych
e,5,6,13,14,11,12
f,brak danych,brak danych,brak danych,brak danych,16,17


In [172]:
help(DataFrame.join)

Help on function join in module pandas.core.frame:

join(self, other, on=None, how='left', lsuffix='', rsuffix='', sort=False)
    Join columns with other DataFrame either on index or on a key
    column. Efficiently Join multiple DataFrame objects by index at once by
    passing a list.
    
    Parameters
    ----------
    other : DataFrame, Series with name field set, or list of DataFrame
        Index should be similar to one of the columns in this one. If a
        Series is passed, its name attribute must be set, and that will be
        used as the column name in the resulting joined DataFrame
    on : column name, tuple/list of column names, or array-like
        Column(s) in the caller to join on the index in other,
        otherwise joins index-on-index. If multiples
        columns given, the passed DataFrame must have a MultiIndex. Can
        pass an array as the join key if not already contained in the
        calling DataFrame. Like an Excel VLOOKUP operation
    how : 

# Concatenating Along an Axis

Another kind of data combination operation is alternatively referred to as concatenation,
binding, or stacking. NumPy has a concatenate function for doing this with raw
NumPy arrays:

page 185

In [6]:
arr = np.arange(12).reshape((3,4))

arr

np.concatenate([arr,arr], axis = 1)
np.concatenate([arr,arr], axis = 0)

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

## Concat in Pandas
In the context of pandas objects such as Series and DataFrame, having labeled axes
enable you to further generalize array concatenation. In particular, you have a number
of additional things to think about:
<br>• If the objects are indexed differently on the other axes, should the collection of
axes be unioned or intersected?
<br>• Do the groups need to be identifiable in the resulting object?
<br>• Does the concatenation axis matter at all?
<br>

The concat function in pandas provides a consistent way to address each of these concerns.
I’ll give a number of examples to illustrate how it works. Suppose we have three
Series with no index overlap. Calling concat with these object in a list glues together the values and indexes:

In [11]:
s1 = Series([0,1], index = ['a', 'b'])
s2 = Series([2,3,4], index = ['c', 'd', 'e'])
s3 = Series([5,6], index = ['f','g'])
s1
s2
s3
pd.concat([s1,s2, s3])

a    0
b    1
dtype: int64

c    2
d    3
e    4
dtype: int64

f    5
g    6
dtype: int64

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

By default concat works along axis=0, producing another Series. If you pass axis=1, the
result will instead be a DataFrame (axis=1 is the columns):

In [12]:
pd.concat([s1,s2,s3], axis = 1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In this case there is no overlap on the other axis, which as you can see is the sorted
union (the 'outer' join) of the indexes. You can instead intersect them by passing
```python
join='inner'
```
<br>
You can even specify the axes to be used on the other axes with 
```python
join_axes
```

In [20]:
s4 = pd.concat([s1*5, s3])
s4
s1
pd.concat([s1, s4], axis = 1)
pd.concat([s1, s4], axis = 1, join = 'inner') # concatenate by indexes which are in both Series
pd.concat([s1, s4], axis = 1, join_axes=[['a','b','c', 'e']])

a    0
b    5
f    5
g    6
dtype: int64

a    0
b    1
dtype: int64

,0,1
a,0.0,0
b,1.0,5
f,NaN,5
g,NaN,6


,0,1
a,0,0
b,1,5


,0,1
a,0.0,0.0
b,1.0,5.0
c,NaN,NaN
e,NaN,NaN


One issue is that the concatenated pieces are not identifiable in the result. Suppose
instead you wanted to create a hierarchical index on the concatenation axis. To do this,
use the keys argument:

In [25]:
result = pd.concat([s1,s1,s3], keys = ['s1', 's1copy', 's3'])
s1
s3
result

# Much more on the unstack function later
result.unstack(fill_value = "missing")

help(Series.unstack)

a    0
b    1
dtype: int64

f    5
g    6
dtype: int64

s1      a    0
        b    1
s1copy  a    0
        b    1
s3      f    5
        g    6
dtype: int64

,a,b,f,g
s1,0,1,missing,missing
s1copy,0,1,missing,missing
s3,missing,missing,5,6


Help on function unstack in module pandas.core.series:

unstack(self, level=-1, fill_value=None)
    Unstack, a.k.a. pivot, Series with MultiIndex to produce DataFrame.
    The level involved will automatically get sorted.
    
    Parameters
    ----------
    level : int, string, or list of these, default last level
        Level(s) to unstack, can pass level name
    fill_value : replace NaN with this value if the unstack produces
        missing values
    
        .. versionadded: 0.18.0
    
    Examples
    --------
    >>> s = pd.Series([1, 2, 3, 4],
    ...     index=pd.MultiIndex.from_product([['one', 'two'], ['a', 'b']]))
    >>> s
    one  a    1
         b    2
    two  a    3
         b    4
    dtype: int64
    
    >>> s.unstack(level=-1)
         a  b
    one  1  2
    two  3  4
    
    >>> s.unstack(level=0)
       one  two
    a    1    3
    b    2    4
    
    Returns
    -------
    unstacked : DataFrame



In the case of combining Series along axis=1, the keys become the DataFrame column
headers.
The same logic extends to DataFrame objects.

In [30]:
#Series
pd.concat([s1, s2, s3], axis = 1, keys= ['one', 'two', 'three'])

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [49]:
#DataFrame
df1 = DataFrame(np.arange(6).reshape(3,2), index=['a', 'b', 'c'], columns=['one', 'two'])
# %timeit df2 = DataFrame(5+ np.arange(4).reshape(2, 2), index=['a', 'c'], columns=['three', 'four'])
df2 = DataFrame(np.arange(5, 9).reshape(2, 2), index=['a', 'c'], columns=['three', 'four'])
df1
df2

pd.concat([df1, df2], axis =1, keys = ['level1', 'level2'])
#If you pass a dict of objects instead of a list, the dict’s keys will be used for the keys option:
pd.concat({'level1': df1, 'level2': df2}, axis=1)
pd.concat([df1, df2], axis =1)



,one,two
a,0,1
b,2,3
c,4,5


,three,four
a,5,6
c,7,8


level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

,one,two,three,four
a,0,1,5.0,6.0
b,2,3,NaN,NaN
c,4,5,7.0,8.0


In [ ]:
help(pd.concat)

In [50]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'], names=['upper', 'lower'])


upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

In [52]:
df1 = DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df2 = DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])
df1
df2
#DataFrames in which the row index is not meaningful in the context of the analysis you can ignore it:
pd.concat([df1, df2], ignore_index=False)
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,0.283365,-0.164251,-0.974816,0.248963
1,-0.187580,-0.968880,-1.217584,-0.046259
2,0.689879,-0.453716,0.390974,-0.049731


,b,d,a
0,0.421138,1.286992,-0.604259
1,-1.227938,-0.914241,-0.254769


,a,b,c,d
0,0.283365,-0.164251,-0.974816,0.248963
1,-0.187580,-0.968880,-1.217584,-0.046259
2,0.689879,-0.453716,0.390974,-0.049731
0,-0.604259,0.421138,NaN,1.286992
1,-0.254769,-1.227938,NaN,-0.914241


,a,b,c,d
0,0.283365,-0.164251,-0.974816,0.248963
1,-0.187580,-0.968880,-1.217584,-0.046259
2,0.689879,-0.453716,0.390974,-0.049731
3,-0.604259,0.421138,NaN,1.286992
4,-0.254769,-1.227938,NaN,-0.914241


## Combining Data with Overlap
Another data combination situation can’t be expressed as either a merge or concatenation
operation. You may have two datasets whose indexes overlap in full or part. As
a motivating example, consider NumPy’s where function, which expressed a vectorized
if-else:

In [57]:
a = Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan], index=['f', 'e', 'd', 'c', 'b', 'a'])
b = Series(np.arange(len(a), dtype=np.float64), index=['f', 'e', 'd', 'c', 'b', 'a'])
b[-1] = np.nan
a
b

np.where(pd.isnull(a), b, a)

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

array([ 0. ,  2.5,  2. ,  3.5,  4.5,  nan])

Series has a 
```python
combine_first 
```
method
, which performs the equivalent of this operation
plus data alignment.
<br>

With DataFrames, combine_first naturally does the same thing column by column, so
you can think of it as “patching” missing data in the calling object with data from the
object you pass:

In [64]:
help(Series.combine_first)

Help on function combine_first in module pandas.core.series:

combine_first(self, other)
    Combine Series values, choosing the calling Series's values
    first. Result index will be the union of the two indexes
    
    Parameters
    ----------
    other : Series
    
    Returns
    -------
    y : Series



In [70]:
#always taking b as first one, in case of NaN in b taking a
a['d'] = 21
a[2:]
b['c'] = NaN
b[:-2]
b[:-2].combine_first(a[2:])

df1 = DataFrame({'a': [1., np.nan, 5., np.nan],
    'b': [np.nan, 2., np.nan, 6.],
    'c': range(2, 18, 4)})
df2 = DataFrame({'a': [5., 4., np.nan, 3., 7.],
    'b': [np.nan, 3., 4., 6., 8.]})
df1
df2
df1.combine_first(df2)

d    21.0
c     3.5
b     4.5
a     NaN
dtype: float64

f    0.0
e    1.0
d    2.0
c    NaN
dtype: float64

a    NaN
b    4.5
c    3.5
d    2.0
e    1.0
f    0.0
dtype: float64

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


**=========================================================================================================================**

**=========================================================================================================================**